## 10-3-1 visdom example

In [39]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets

### terminal : python -m visdom.server
![title](t.png)

In [40]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


### text
![title](h.png)

In [3]:
vis.text("Hello, world!",env="main")

'window_38369209cdb5f2'

### image
![title](pi.png)

In [4]:
#평균이 0이고 표준편차가 1인 가우시안 정규분포를 이용해 생성
a=torch.randn(3,200,200) 
vis.image(a)

'window_3836920bef0a26'

![title](i.png)

In [5]:
vis.images(torch.Tensor(3,3,28,28)) # 0으로 채워짐

'window_383692106a327c'

### CIFAR10 : training을 위한 데이터셋

In [6]:
MNIST = dsets.MNIST(root="./MNIST_data",train = True,transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10",train = True, transform=torchvision.transforms.ToTensor(),download=True)

Files already downloaded and verified


![title](c.png)

In [7]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([3, 32, 32])


'window_383692285a2522'

![title](dh.png)

In [8]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([1, 28, 28])


'window_3836922d1c877a'

In [9]:
data_loader = torch.utils.data.DataLoader(dataset = MNIST,
                                          batch_size = 32,
                                          shuffle = False)

![title](v.png)

In [10]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [11]:
vis.close(env="main") # 열려있던 창 다 닫힘

''

### line
![title](y.png)
![title](da.png)

In [12]:
Y_data = torch.randn(5)
plt = vis.line (Y=Y_data)

![title](xy.png)

In [13]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

![title](a2.png)
![title](a.png)

In [14]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_38369258f7fed2'

![title](yyy.png)
![title](yy.png)

In [15]:
num = torch.Tensor(list(range(0,10)))
num = num.view(-1,1)
num = torch.cat((num,num),dim=1)

y = torch.randn(10,2)

plt = vis.line(Y=y, X = num)

![title](t1.png)

In [16]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend=True))

In [17]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', legend = ['1번'],showlegend=True))

In [18]:
plt = vis.line(Y=torch.randn(10,2), X = num, opts=dict(title='Test', legend=['1번','2번'],showlegend=True))

### update
- loss_value, num : tensor
![title](up.png)

In [57]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [29]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

## 10-3-2 MNIST-CNN with visdom

In [41]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [42]:
vis.close(env="main")

''

In [43]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [44]:
#torch.manual_seed(777)
#if device =='cuda':
    #torch.cuda.manual_seed_all(777)

In [45]:
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [46]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform = transforms.ToTensor(),
                        download=True)

In [47]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle =True,
                                          drop_last=True)

![title](cnn2.png)

In [52]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [53]:
model = CNN().to(device)

value = (torch.Tensor(1,1,28,28)).to(device)
print( (model(value)).shape )

torch.Size([1, 10])


In [54]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

### make plot
![title](l2.png)

In [55]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

![title](ra2.png)
![title](ra.png)

In [58]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}]\tcost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')

[Epoch:1]	cost = 0.04011575132608414
[Epoch:2]	cost = 0.02841002680361271
[Epoch:3]	cost = 0.0213644839823246
[Epoch:4]	cost = 0.01861419901251793
[Epoch:5]	cost = 0.014173579402267933
[Epoch:6]	cost = 0.012169653549790382
[Epoch:7]	cost = 0.009523596614599228
[Epoch:8]	cost = 0.011066339910030365
[Epoch:9]	cost = 0.007608357351273298
[Epoch:10]	cost = 0.007567554246634245
[Epoch:11]	cost = 0.008478053845465183
[Epoch:12]	cost = 0.005649091210216284
[Epoch:13]	cost = 0.007472857367247343
[Epoch:14]	cost = 0.007814520969986916
[Epoch:15]	cost = 0.005996528081595898
Learning Finished!


![title](a3.png)

In [42]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())

C:\Users\Moon\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\Moon\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.975600004196167
